# LOO

## import libraries

In [1]:
import os
import pickle
import random
from glob import glob
from time import sleep

import numpy as np
import pandas as pd
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from albumentations import (CenterCrop, Compose, HorizontalFlip, Normalize,
                            RandomCrop, VerticalFlip)
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, RandomSampler
from tqdm import tqdm

from clmodel.dataset import CLModelDataset
from clmodel.evaluate import Metrics, macro_auroc, macro_balanced_accuracy
from clmodel.model import FrozenEffnetB4Model
from clmodel.train import train_loop
from clmodel.utils import AverageValue, Logger, fix_seed

## set seeds

In [2]:
seed = 123
fix_seed(seed)


def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


g = torch.Generator()
g.manual_seed(seed)


## load data

In [3]:
df = pd.read_csv("../../data/TGGATEs/processed/train_val_for_model_training.csv")


In [4]:
df["path"] = df["path"].str.replace("HDD", "extHDD1")

In [5]:
ft_list = list(df.columns[3:11])
ft_list


['Proliferation, bile duct',
 'Ground glass appearance',
 'Increased mitosis',
 'Inclusion body, intracytoplasmic',
 'Deposit, pigment',
 'Single cell necrosis',
 'Vacuolization, cytoplasmic',
 'Swelling']

## image preprocessing

In [6]:
image_size = 512

tr_transform = Compose(
    [
        # RandomCrop(image_size, image_size),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        Normalize(),
        ToTensorV2(),
    ]
)
vl_transform = Compose(
    [Normalize(), ToTensorV2()]  # CenterCrop(image_size, image_size),
)


## create image_dict

In [7]:
n_epochs = 5

tr = df[df["fold"] != 2]
vl = df[df["fold"] == 2]

ft_dict = {v[0]: v[1:] for v in df[["path"] + ft_list].to_numpy()}

train_dataset = CLModelDataset(
    tr["path"].values, None, transform=RandomCrop(image_size, image_size)
)
train_loader = DataLoader(
    train_dataset,
    num_workers=4,
    batch_size=16,
    sampler=RandomSampler(
        train_dataset, num_samples=len(train_dataset) // 10, replacement=True
    ),
    pin_memory=True,
    drop_last=True,
    worker_init_fn=seed_worker,
    generator=g,
)

new_tr = []
image_dict = {}
count = 0
for epoch in range(n_epochs):
    for x, pathes in tqdm(train_loader):
        for i in range(16):
            image_dict[f"dummy/dummy/{count}"] = x[i].numpy()
            new_tr.append([f"dummy/dummy/{count}"] + list(ft_dict[pathes[i]]))
            count += 1
    # with open("../../temp/cache_image_dict_new_tr.pickle", "wb") as f:
    #     pickle.dump((image_dict, new_tr), f)
new_tr = pd.DataFrame(new_tr, columns=["path"] + ft_list)


100%|██████████| 321/321 [15:58<00:00,  2.99s/it]


In [8]:
valid_dataset = CLModelDataset(
    vl["path"].values,
    None,
    transform=CenterCrop(image_size, image_size),
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=32,
    drop_last=False,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
)
new_vl = []
for x, pathes in tqdm(valid_loader):
    for i in range(len(x)):
        image_dict[f"dummy/dummy/{count}"] = x[i].numpy()
        new_vl.append([f"dummy/dummy/{count}"] + list(ft_dict[pathes[i]]))
        count += 1
new_vl = pd.DataFrame(new_vl, columns=["path"] + ft_list)


100%|██████████| 536/536 [1:19:43<00:00,  8.92s/it]


In [9]:
new_vl

,path,"Proliferation, bile duct",Ground glass appearance,Increased mitosis,"Inclusion body, intracytoplasmic","Deposit, pigment",Single cell necrosis,"Vacuolization, cytoplasmic",Swelling
0,dummy/dummy/25680,0.000000,0.999631,0.000000,0.0,0.0,0.0,0.0,0.000000
1,dummy/dummy/25681,0.000000,0.999841,0.000000,0.0,0.0,0.0,0.0,0.000000
2,dummy/dummy/25682,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
3,dummy/dummy/25683,0.999986,0.000000,0.000000,0.0,0.0,0.0,0.0,0.998901
4,dummy/dummy/25684,0.000000,0.000000,0.978140,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...
17116,dummy/dummy/42796,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
17117,dummy/dummy/42797,0.938293,0.000000,0.000000,0.0,0.0,0.0,0.0,0.971595
17118,dummy/dummy/42798,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
17119,dummy/dummy/42799,0.000000,0.000000,0.908885,0.0,0.0,0.0,0.0,0.000000


In [10]:
new_vl = pd.DataFrame(new_vl[:len(vl)], columns=["path"] + ft_list)


## train

In [11]:
for ft in ft_list:
    print(f"==============={ft}=============")
    loo_ft_list = [f for f in ft_list if f != ft]
    train_dataset = CLModelDataset(
        new_tr["path"].values,
        new_tr[loo_ft_list].values >= 0.5,
        image_dict,
        transform=tr_transform,
        length=len(tr) // 10,
        cache_mode=True,
    )
    valid_dataset = CLModelDataset(
        new_vl["path"].values,
        new_vl[loo_ft_list].values >= 0.5,
        image_dict,
        transform=vl_transform,
    )

    train_loader = DataLoader(
        train_dataset,
        num_workers=4,
        batch_size=16,
        shuffle=False,
        pin_memory=True,
        drop_last=False,
    )
    valid_loader = DataLoader(
        valid_dataset,
        num_workers=4,
        batch_size=32,
        shuffle=False,
        pin_memory=True,
        drop_last=False,
    )

    criterion = nn.BCEWithLogitsLoss()

    n_epochs = 5

    out_dir = f"../../outputs/230305TGGATEs_ft_loo_{ft}_seed123_epoch5"
    os.system(f'mkdir "{out_dir}"')

    for depth in range(9):
        print(f"=================Depth {depth}===================")
        if depth >= 8:
            model = timm.create_model(
                "tf_efficientnet_b4_ns", pretrained=True, num_classes=len(loo_ft_list)
            )
            if depth == 9:
                depth = "8_10epochs"
                n_epochs = 10
        else:
            model = FrozenEffnetB4Model(depth, len(loo_ft_list))

        model.to("cuda")
        optimizer = optim.Adam(model.parameters(), lr=5e-4)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 10, 1e-6)

        metrics = [
            Metrics("macro AUROC", macro_auroc, "+"),
            Metrics("macro balanced accuracy", macro_balanced_accuracy, "+"),
        ]
        os.system(f'mkdir "{out_dir}/{depth}"')
        res = train_loop(
            model,
            train_loader,
            valid_loader,
            0,
            criterion,
            optimizer,
            "cuda",
            n_epochs,
            scheduler,
            metrics,
            f"{out_dir}/{depth}",
            f"effnetb4_freeze{depth}",
            preprocess=lambda x: x.sigmoid(),
            verbose=100,
            logger=Logger(),
        )

        with open(f"{out_dir}/{depth}/result.pickle", "wb") as f:
            pickle.dump(res, f)
        sleep(100)

===============Proliferation, bile duct=============
=================Depth 0===================


mkdir: cannot create directory ‘../../outputs/230305TGGATEs_ft_loo_Proliferation, bile duct_seed123_epoch5’: File exists


Epoch 1


mkdir: cannot create directory ‘../../outputs/230305TGGATEs_ft_loo_Proliferation, bile duct_seed123_epoch5/0’: File exists


Step: 1/321 Loss: 0.6976 Elapsed time 2.4 Rest time 763.4
Step: 101/321 Loss: 0.2761 Elapsed time 9.8 Rest time 21.4
Step: 201/321 Loss: 0.2403 Elapsed time 16.3 Rest time 9.7
Step: 301/321 Loss: 0.2227 Elapsed time 20.6 Rest time 1.4
Step: 321/321 Loss: 0.2203 Elapsed time 21.5 Rest time 0.0
Step: 1/536 Loss: 0.2416 Elapsed time 0.8 Rest time 423.0
Step: 101/536 Loss: 0.2084 Elapsed time 16.0 Rest time 68.7
Step: 201/536 Loss: 0.2048 Elapsed time 30.9 Rest time 51.5
Step: 301/536 Loss: 0.2053 Elapsed time 46.5 Rest time 36.3
Step: 401/536 Loss: 0.2054 Elapsed time 61.1 Rest time 20.6
Step: 501/536 Loss: 0.2048 Elapsed time 75.7 Rest time 5.3
Step: 536/536 Loss: 0.2048 Elapsed time 80.6 Rest time 0.0
macro AUROC : 0.8444
This is best macro AUROC.
saved model.
macro balanced accuracy : 0.5971
This is best macro balanced accuracy.
saved model.
loss : 0.2048
This is best loss.
saved model.
Epoch 2
Step: 1/321 Loss: 0.1678 Elapsed time 0.5 Rest time 158.6
Step: 101/321 Loss: 0.1783 Elapsed

...

Step: 1/321 Loss: 0.6930 Elapsed time 0.7 Rest time 211.6
Step: 101/321 Loss: 0.2482 Elapsed time 21.6 Rest time 47.1
Step: 201/321 Loss: 0.2126 Elapsed time 38.1 Rest time 22.7
Step: 301/321 Loss: 0.1953 Elapsed time 54.5 Rest time 3.6
Step: 321/321 Loss: 0.1927 Elapsed time 59.3 Rest time 0.0
Step: 1/536 Loss: 0.1826 Elapsed time 1.0 Rest time 549.6
Step: 101/536 Loss: 0.1561 Elapsed time 15.6 Rest time 67.0
Step: 201/536 Loss: 0.1560 Elapsed time 30.1 Rest time 50.2
Step: 301/536 Loss: 0.1570 Elapsed time 44.7 Rest time 34.9
Step: 401/536 Loss: 0.1569 Elapsed time 59.3 Rest time 20.0
Step: 501/536 Loss: 0.1563 Elapsed time 73.9 Rest time 5.2
Step: 536/536 Loss: 0.1553 Elapsed time 79.0 Rest time 0.0
macro AUROC : 0.887
This is best macro AUROC.
saved model.
macro balanced accuracy : 0.7083
This is best macro balanced accuracy.
saved model.
loss : 0.1553
This is best loss.
saved model.
Epoch 2
Step: 1/321 Loss: 0.1512 Elapsed time 0.9 Rest time 286.4
Step: 101/321 Loss: 0.1323 Elapse

mkdir: cannot create directory ‘../../outputs/230305TGGATEs_ft_loo_Swelling_seed123_epoch5/7’: File exists


Step: 1/321 Loss: 0.7005 Elapsed time 0.8 Rest time 249.1
Step: 101/321 Loss: 0.2530 Elapsed time 22.6 Rest time 49.3
Step: 201/321 Loss: 0.2176 Elapsed time 40.7 Rest time 24.3
Step: 301/321 Loss: 0.1976 Elapsed time 62.0 Rest time 4.1
Step: 321/321 Loss: 0.1951 Elapsed time 66.3 Rest time 0.0
Step: 1/536 Loss: 0.1564 Elapsed time 0.6 Rest time 344.8
Step: 101/536 Loss: 0.1483 Elapsed time 15.2 Rest time 65.3
Step: 201/536 Loss: 0.1469 Elapsed time 29.7 Rest time 49.5
Step: 301/536 Loss: 0.1474 Elapsed time 44.3 Rest time 34.6
Step: 401/536 Loss: 0.1485 Elapsed time 58.9 Rest time 19.8
Step: 501/536 Loss: 0.1477 Elapsed time 73.4 Rest time 5.1
Step: 536/536 Loss: 0.1475 Elapsed time 78.4 Rest time 0.0
macro AUROC : 0.8956
This is best macro AUROC.
saved model.
macro balanced accuracy : 0.6616
This is best macro balanced accuracy.
saved model.
loss : 0.1475
This is best loss.
saved model.
Epoch 2
Step: 1/321 Loss: 0.1625 Elapsed time 1.1 Rest time 351.8
Step: 101/321 Loss: 0.1360 Elaps

mkdir: cannot create directory ‘../../outputs/230305TGGATEs_ft_loo_Swelling_seed123_epoch5/8’: File exists


Step: 1/321 Loss: 0.8163 Elapsed time 0.7 Rest time 217.4
Step: 101/321 Loss: 0.2567 Elapsed time 18.9 Rest time 41.3
Step: 201/321 Loss: 0.2139 Elapsed time 41.3 Rest time 24.7
Step: 301/321 Loss: 0.1934 Elapsed time 60.1 Rest time 4.0
Step: 321/321 Loss: 0.1912 Elapsed time 63.8 Rest time 0.0
Step: 1/536 Loss: 0.2074 Elapsed time 0.6 Rest time 345.3
Step: 101/536 Loss: 0.1604 Elapsed time 15.2 Rest time 65.4
Step: 201/536 Loss: 0.1611 Elapsed time 29.8 Rest time 49.6
Step: 301/536 Loss: 0.2064 Elapsed time 44.3 Rest time 34.6
Step: 401/536 Loss: 0.1955 Elapsed time 58.9 Rest time 19.8
Step: 501/536 Loss: 0.2026 Elapsed time 73.5 Rest time 5.1
Step: 536/536 Loss: 0.1992 Elapsed time 78.4 Rest time 0.0
macro AUROC : 0.877
This is best macro AUROC.
saved model.
macro balanced accuracy : 0.6559
This is best macro balanced accuracy.
saved model.
loss : 0.1992
This is best loss.
saved model.
Epoch 2
Step: 1/321 Loss: 0.1148 Elapsed time 0.7 Rest time 222.2
Step: 101/321 Loss: 0.1310 Elapse

## train with the all pathological findings

In [12]:
train_dataset = CLModelDataset(
    new_tr["path"].values,
    new_tr[ft_list].values >= 0.5,
    image_dict,
    transform=tr_transform,
    length=len(tr) // 10,
    cache_mode=True,
)
valid_dataset = CLModelDataset(
    new_vl["path"].values,
    new_vl[ft_list].values >= 0.5,
    image_dict,
    transform=vl_transform,
)

train_loader = DataLoader(
    train_dataset,
    num_workers=4,
    batch_size=16,
    shuffle=False,
    pin_memory=True,
    drop_last=False,
)
valid_loader = DataLoader(
    valid_dataset,
    num_workers=4,
    batch_size=32,
    shuffle=False,
    pin_memory=True,
    drop_last=False,
)

criterion = nn.BCEWithLogitsLoss()

n_epochs = 5

out_dir = "../../outputs/230305TGGATEs_model_seed123_epoch5"
os.system(f'mkdir "{out_dir}"')

for depth in range(9):
    print(f"=================Depth {depth}===================")
    if depth >= 8:
        model = timm.create_model(
            "tf_efficientnet_b4_ns", pretrained=True, num_classes=len(ft_list)
        )
        if depth == 9:
            depth = "8_10epochs"
            n_epochs = 10
    else:
        model = FrozenEffnetB4Model(depth, len(ft_list))

    model.to("cuda")
    optimizer = optim.Adam(model.parameters(), lr=5e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 10, 1e-6)

    metrics = [
        Metrics("macro AUROC", macro_auroc, "+"),
        Metrics("macro balanced accuracy", macro_balanced_accuracy, "+"),
    ]
    os.system(f'mkdir "{out_dir}/{depth}"')
    res = train_loop(
        model,
        train_loader,
        valid_loader,
        0,
        criterion,
        optimizer,
        "cuda",
        n_epochs,
        scheduler,
        metrics,
        f"{out_dir}/{depth}",
        f"effnetb4_freeze{depth}",
        preprocess=lambda x: x.sigmoid(),
        verbose=100,
        logger=Logger(),
    )

    with open(f"{out_dir}/{depth}/result.pickle", "wb") as f:
        pickle.dump(res, f)
    sleep(100)

mkdir: cannot create directory ‘../../outputs/230305TGGATEs_model_seed123_epoch5’: File exists


=================Depth 0===================
Epoch 1


mkdir: cannot create directory ‘../../outputs/230305TGGATEs_model_seed123_epoch5/0’: File exists


Step: 1/321 Loss: 0.6857 Elapsed time 0.5 Rest time 169.4
Step: 101/321 Loss: 0.2814 Elapsed time 6.7 Rest time 14.5
Step: 201/321 Loss: 0.2464 Elapsed time 14.8 Rest time 8.9
Step: 301/321 Loss: 0.2283 Elapsed time 22.8 Rest time 1.5
Step: 321/321 Loss: 0.2259 Elapsed time 24.4 Rest time 0.0
Step: 1/536 Loss: 0.2228 Elapsed time 1.0 Rest time 523.1
Step: 101/536 Loss: 0.1986 Elapsed time 15.5 Rest time 66.7
Step: 201/536 Loss: 0.1956 Elapsed time 30.0 Rest time 50.0
Step: 301/536 Loss: 0.1959 Elapsed time 44.6 Rest time 34.8
Step: 401/536 Loss: 0.1961 Elapsed time 59.2 Rest time 19.9
Step: 501/536 Loss: 0.1953 Elapsed time 73.9 Rest time 5.2
Step: 536/536 Loss: 0.1951 Elapsed time 79.0 Rest time 0.0
macro AUROC : 0.8423
This is best macro AUROC.
saved model.
macro balanced accuracy : 0.6019
This is best macro balanced accuracy.
saved model.
loss : 0.1951
This is best loss.
saved model.
Epoch 2
Step: 1/321 Loss: 0.1701 Elapsed time 0.5 Rest time 158.4
Step: 101/321 Loss: 0.1820 Elapsed

mkdir: cannot create directory ‘../../outputs/230305TGGATEs_model_seed123_epoch5/1’: File exists


...

Step: 1/321 Loss: 0.7001 Elapsed time 0.7 Rest time 210.7
Step: 101/321 Loss: 0.2417 Elapsed time 21.5 Rest time 46.8
Step: 201/321 Loss: 0.2073 Elapsed time 39.0 Rest time 23.3
Step: 301/321 Loss: 0.1893 Elapsed time 58.9 Rest time 3.9
Step: 321/321 Loss: 0.1871 Elapsed time 62.2 Rest time 0.0
Step: 1/536 Loss: 0.1849 Elapsed time 0.7 Rest time 351.2
Step: 101/536 Loss: 0.1638 Elapsed time 15.2 Rest time 65.4
Step: 201/536 Loss: 0.1611 Elapsed time 29.8 Rest time 49.6
Step: 301/536 Loss: 0.1616 Elapsed time 44.3 Rest time 34.6
Step: 401/536 Loss: 0.1616 Elapsed time 58.9 Rest time 19.8
Step: 501/536 Loss: 0.1608 Elapsed time 73.6 Rest time 5.1
Step: 536/536 Loss: 0.1603 Elapsed time 78.6 Rest time 0.0
macro AUROC : 0.8981
This is best macro AUROC.
saved model.
macro balanced accuracy : 0.6966
This is best macro balanced accuracy.
saved model.
loss : 0.1603
This is best loss.
saved model.
Epoch 2
Step: 1/321 Loss: 0.1416 Elapsed time 0.7 Rest time 212.9
Step: 101/321 Loss: 0.1319 Elaps

mkdir: cannot create directory ‘../../outputs/230305TGGATEs_model_seed123_epoch5/7’: File exists


Step: 1/321 Loss: 0.6965 Elapsed time 0.7 Rest time 218.1
Step: 101/321 Loss: 0.2456 Elapsed time 22.8 Rest time 49.7
Step: 201/321 Loss: 0.2144 Elapsed time 40.8 Rest time 24.4
Step: 301/321 Loss: 0.1947 Elapsed time 61.3 Rest time 4.1
Step: 321/321 Loss: 0.1923 Elapsed time 66.4 Rest time 0.0
Step: 1/536 Loss: 0.1672 Elapsed time 0.6 Rest time 338.6
Step: 101/536 Loss: 0.1734 Elapsed time 15.2 Rest time 65.3
Step: 201/536 Loss: 0.1700 Elapsed time 29.7 Rest time 49.5
Step: 301/536 Loss: 0.1704 Elapsed time 44.3 Rest time 34.6
Step: 401/536 Loss: 0.1711 Elapsed time 58.8 Rest time 19.8
Step: 501/536 Loss: 0.1707 Elapsed time 73.4 Rest time 5.1
Step: 536/536 Loss: 0.1703 Elapsed time 78.4 Rest time 0.0
macro AUROC : 0.8938
This is best macro AUROC.
saved model.
macro balanced accuracy : 0.6908
This is best macro balanced accuracy.
saved model.
loss : 0.1703
This is best loss.
saved model.
Epoch 2
Step: 1/321 Loss: 0.1531 Elapsed time 0.8 Rest time 259.4
Step: 101/321 Loss: 0.1354 Elaps

mkdir: cannot create directory ‘../../outputs/230305TGGATEs_model_seed123_epoch5/8’: File exists


Step: 1/321 Loss: 0.9987 Elapsed time 0.6 Rest time 205.9
Step: 101/321 Loss: 0.2449 Elapsed time 24.2 Rest time 52.6
Step: 201/321 Loss: 0.2064 Elapsed time 49.2 Rest time 29.4
Step: 301/321 Loss: 0.1880 Elapsed time 71.5 Rest time 4.8
Step: 321/321 Loss: 0.1853 Elapsed time 76.7 Rest time 0.0
Step: 1/536 Loss: 0.1526 Elapsed time 0.9 Rest time 491.7
Step: 101/536 Loss: 0.1591 Elapsed time 15.5 Rest time 66.7
Step: 201/536 Loss: 0.1568 Elapsed time 30.2 Rest time 50.3
Step: 301/536 Loss: 9.3580 Elapsed time 44.9 Rest time 35.0
Step: 401/536 Loss: 9.0936 Elapsed time 59.4 Rest time 20.0
Step: 501/536 Loss: 9.5321 Elapsed time 74.0 Rest time 5.2
Step: 536/536 Loss: 8.9353 Elapsed time 79.1 Rest time 0.0
macro AUROC : 0.8915
This is best macro AUROC.
saved model.
macro balanced accuracy : 0.6893
This is best macro balanced accuracy.
saved model.
loss : 8.9353
This is best loss.
saved model.
Epoch 2
Step: 1/321 Loss: 0.1043 Elapsed time 0.7 Rest time 215.8
Step: 101/321 Loss: 0.1298 Elaps